# Sample Model For Project Portfolio Selection Multi-Yr Budgeting
# Part 1 : Loading the data from a CSV file

In [1]:
import pandas as pd
proj_list= pd.read_csv('Scenario4_InputData.csv', index_col=['Index'])
proj_list = proj_list.drop(['Year_1','Year_2','Year_3','Year4_Onwards'], axis=1)
discountrate=0.05
#Annual CAPEX Constraints Yr1,2,3,4+ 
capexconstraints=[20,30,25,0] #Note final year 4 onwards is zero as any that do not fit in 1-3 will spill to future capex

In [2]:
proj_list

,Programme,CAPEX,Type,Location,Region,BenefitPerYr
Index,,,,,,
1,CivilStructural,5.0,AssetIntegrity,Facility H,Zone3,2.0
2,Electrical,15.0,AssetIntegrity,Facility M,Zone5,3.0
3,Piping,10.0,AssetIntegrity,Facility E,Zone5,5.0
4,Instrumentation,5.0,AssetIntegrity,Facility E,Zone5,2.0
5,CivilStructural,10.0,Growth,Facility M,Zone5,4.0
6,Expansion,15.0,Growth,Facility C,Zone3,8.0
7,Debottlenecking,10.0,Growth,Facility M,Zone5,2.0
8,Revamp,2.0,Growth,Facility G,Zone1,1.0
9,Revamp,3.0,Growth,Facility F,Zone1,1.0


# Part 2 Solving Non Linear Programming Via Pyomo

In [3]:
from pyomo.environ import *


In [4]:
capex = dict(zip(proj_list.index,proj_list['CAPEX']))
benefitperyr= dict(zip(proj_list.index,proj_list['BenefitPerYr']))

In [5]:
benefitperyr.values()

dict_values([2.0, 3.0, 5.0, 2.0, 4.0, 8.0, 2.0, 1.0, 1.0, 2.0])

In [6]:
model = ConcreteModel()
model.PROJECTS = capex.keys()


In [7]:
model.YearOne = Var( model.PROJECTS, within=Binary)
model.YearTwo = Var( model.PROJECTS, within=Binary)
model.YearThree = Var( model.PROJECTS, within=Binary)
model.YearFour = Var( model.PROJECTS, within=Binary)

In [ ]:
#Initialize Years with random input and view
import numpy as np
for i in model.PROJECTS:
    model.YearOne[i].value=np.random.choice(2)
    model.YearTwo[i].value=np.random.choice(2)
    model.YearThree[i].value=np.random.choice(2)
    model.YearFour[i].value=np.random.choice(2)
    
pyomoInitalisationCheck=pd.DataFrame(columns = ["Yr1", "Yr2", "Yr3", "Yr4_Onwards"])
for i in model.PROJECTS:
    to_append = [model.YearOne[i].value, model.YearTwo[i].value, model.YearThree[i].value, model.YearFour[i].value]
    perprojectyear = pd.Series(to_append, index = pyomoInitalisationCheck.columns)
    pyomoInitalisationCheck = pyomoInitalisationCheck.append(perprojectyear, ignore_index=True)

pyomoInitalisationCheck.index += 1 
pyomoInitalisationCheck

In [8]:
# Naive formulation as NLP with Expr_if

model.objective = Objective(expr=sum(Expr_if(IF=(model.YearOne[i]) > 0.0, THEN= benefitperyr[i]*(1/(1+discountrate)**1 + 1/(1+discountrate)**2 + 1/(1+discountrate)**3), ELSE= 0) for i in model.PROJECTS)
                            + sum(Expr_if(IF=(model.YearTwo[i]) > 0.0, THEN= benefitperyr[i]*(1/(1+discountrate)**2 + 1/(1+discountrate)**3), ELSE= 0) for i in model.PROJECTS)
                            + sum(Expr_if(IF=(model.YearTwo[i]) > 0.0, THEN= benefitperyr[i]*(1/(1+discountrate)**3), ELSE= 0) for i in model.PROJECTS)
                            , sense=maximize)

In [ ]:
'''# If Then using Big M notation

model.year1BigM = Var( model.PROJECTS, domain=Binary)
model.year2BigM = Var( model.PROJECTS, domain=Binary)
model.year3BigM = Var( model.PROJECTS, domain=Binary)
BigM=10000

model.objective = Objective(expr=
                sum(model.year1BigM[i] * benefitperyr[i]*(1/(1+discountrate)**1 + 1/(1+discountrate)**2 + 1/(1+discountrate)**3)  for i in model.PROJECTS)
                +sum(model.year2BigM[i] * benefitperyr[i]*(1/(1+discountrate)**2 + 1/(1+discountrate)**3)  for i in model.PROJECTS)
                +sum(model.year3BigM[i] * benefitperyr[i]*(1/(1+discountrate)**3)  for i in model.PROJECTS)
                ,sense=maximize)


In [ ]:
'''
# Reformulated as simple LP

model.objective = Objective(expr=
                sum(model.YearOne[i]  * benefitperyr[i]*(1/(1+discountrate)**1 + 1/(1+discountrate)**2 + 1/(1+discountrate)**3)  for i in model.PROJECTS)
                +sum(model.YearTwo[i] * benefitperyr[i]*(1/(1+discountrate)**2 + 1/(1+discountrate)**3)  for i in model.PROJECTS)
                +sum(model.YearThree[i] * benefitperyr[i]*(1/(1+discountrate)**3)  for i in model.PROJECTS)
                ,sense=maximize)

In [9]:
model.limits = ConstraintList()

In [ ]:
'''# If Then using Big M notation

for i in model.PROJECTS:
    model.limits.add(model.YearOne[i] -1 <= model.year1BigM[i] * BigM)
    model.limits.add(model.YearTwo[i] -1 <= model.year2BigM[i] * BigM)
    model.limits.add(model.YearThree[i] -1 <= model.year3BigM[i] * BigM)

In [10]:
# Other Constraints

model.limits.add(sum(model.YearOne[i]*capex[i] for i in model.PROJECTS) <= capexconstraints[0])
model.limits.add(sum(model.YearTwo[i]*capex[i] for i in model.PROJECTS) <= capexconstraints[1])
model.limits.add(sum(model.YearThree[i]*capex[i] for i in model.PROJECTS) <= capexconstraints[2])

for i in model.PROJECTS:
    model.limits.add(model.YearOne[i]+model.YearTwo[i]+model.YearThree[i]+model.YearFour[i]==1)    

In [11]:
# solver = SolverFactory("glpk")
# %time solution = solver.solve(model)

solver = SolverFactory('bonmin')
%time solution = solver.solve(model) #use solver.solve(model,tee=True) for more detail

model.display()

Wall time: 3min 42s
Model unknown

  Variables:
    YearOne : Size=10, Index=YearOne_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :   0.0 :     1 : False : False : Binary
          2 :     0 :   0.0 :     1 : False : False : Binary
          3 :     0 :   0.0 :     1 : False : False : Binary
          4 :     0 :   0.0 :     1 : False : False : Binary
          5 :     0 :   0.0 :     1 : False : False : Binary
          6 :     0 :   1.0 :     1 : False : False : Binary
          7 :     0 :   0.0 :     1 : False : False : Binary
          8 :     0 :   1.0 :     1 : False : False : Binary
          9 :     0 :   1.0 :     1 : False : False : Binary
         10 :     0 :   0.0 :     1 : False : False : Binary
    YearTwo : Size=10, Index=YearTwo_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :   1.0 :     1 : False : False : Binary
          2 :     0 :   0.0 :     1 : False : False : Binary
          3 

In [12]:
from pyomo.opt import SolverStatus, TerminationCondition
if (solution.solver.status == SolverStatus.ok) and (solution.solver.termination_condition == TerminationCondition.optimal):
    print("Solution is feasible and optimal")
    #print("Objective function value = ",  model.value())
elif solution.solver.termination_condition == TerminationCondition.infeasible:
    print ("Failed to find solution.")
else:
    # something else is wrong
    print(str(solution.solver))
    
    
solution.write()
print("\nDisplaying Solution\n" + '-')
model.YearOne.display()
model.YearTwo.display()
model.YearThree.display()
model.YearFour.display()

Solution is feasible and optimal
# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 0
  Number of variables: 40
  Sense: unknown
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Message: bonmin\x3a Optimal
  Termination condition: optimal
  Id: 3
  Error rc: 0
  Time: 220.986967086792
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------------------------------
Solution: 
- number of solutions: 0
  number of solutions displayed: 

In [13]:
pyomosolution=pd.DataFrame(columns = ["Yr1", "Yr2", "Yr3", "Yr4_Onwards","CAPEXYr1","CAPEXYr2","CAPEXYr3","CAPEXYr4Onwards"])
for i in model.PROJECTS:
    to_append = [model.YearOne[i].value,model.YearTwo[i].value,model.YearThree[i].value,model.YearFour[i].value,model.YearOne[i].value*proj_list.loc[i]["CAPEX"],model.YearTwo[i].value*proj_list.loc[i]["CAPEX"],model.YearThree[i].value*proj_list.loc[i]["CAPEX"],model.YearFour[i].value*proj_list.loc[i]["CAPEX"]]
    perprojectyear = pd.Series(to_append, index = pyomosolution.columns)
    pyomosolution = pyomosolution.append(perprojectyear, ignore_index=True)

pyomosolution.index += 1 
pyomosolution

pyomooutput = pd.concat([proj_list, pyomosolution], axis=1)
pyomooutput['NPV_Benefit_3YrHorizon'] = (pyomooutput["Yr1"]*pyomooutput["BenefitPerYr"]*(1/(1+discountrate)**1 + 1/(1+discountrate)**2 + 1/(1+discountrate)**3) + pyomooutput["Yr2"]*pyomooutput["BenefitPerYr"]*(1/(1+discountrate)**2 + 1/(1+discountrate)**3) + pyomooutput["Yr3"]*pyomooutput["BenefitPerYr"]*(1/(1+discountrate)**3))
pyomooutput['NPV_Benefit_3YrHorizon']=pyomooutput['NPV_Benefit_3YrHorizon'].astype(float).round(2)
#pyomooutput=pyomooutput.drop(['Programme', 'Type','Location','Region'], axis=1)
pyomooutput

,Programme,CAPEX,Type,Location,Region,BenefitPerYr,Yr1,Yr2,Yr3,Yr4_Onwards,CAPEXYr1,CAPEXYr2,CAPEXYr3,CAPEXYr4Onwards,NPV_Benefit_3YrHorizon
1,CivilStructural,5.0,AssetIntegrity,Facility H,Zone3,2.0,0.0,1.0,0.0,0.0,0.0,5.0,0.0,0.0,3.54
2,Electrical,15.0,AssetIntegrity,Facility M,Zone5,3.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,15.0,0.00
3,Piping,10.0,AssetIntegrity,Facility E,Zone5,5.0,0.0,1.0,0.0,0.0,0.0,10.0,0.0,0.0,8.85
4,Instrumentation,5.0,AssetIntegrity,Facility E,Zone5,2.0,0.0,1.0,0.0,0.0,0.0,5.0,0.0,0.0,3.54
5,CivilStructural,10.0,Growth,Facility M,Zone5,4.0,0.0,1.0,0.0,0.0,0.0,10.0,0.0,0.0,7.08
6,Expansion,15.0,Growth,Facility C,Zone3,8.0,1.0,0.0,0.0,0.0,15.0,0.0,0.0,0.0,21.79
7,Debottlenecking,10.0,Growth,Facility M,Zone5,2.0,0.0,0.0,1.0,0.0,0.0,0.0,10.0,0.0,1.73
8,Revamp,2.0,Growth,Facility G,Zone1,1.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,2.72
9,Revamp,3.0,Growth,Facility F,Zone1,1.0,1.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,2.72
10,Expansion,7.0,Growth,Facility J,Zone4,2.0,0.0,0.0,1.0,0.0,0.0,0.0,7.0,0.0,1.73


In [14]:
CAPEX_Totals_pyomo=[pyomooutput['CAPEXYr1'].sum(),pyomooutput['CAPEXYr2'].sum(),pyomooutput['CAPEXYr3'].sum(),pyomooutput['CAPEXYr4Onwards'].sum()]

CAPEX_CheckSum_pyomo= pd.DataFrame()
CAPEX_CheckSum_pyomo['Year'] = [1,2,3,4]
CAPEX_CheckSum_pyomo['CAPEX Phasing'] = CAPEX_Totals_pyomo
CAPEX_CheckSum_pyomo['Constraints'] = capexconstraints
print('Maximised Benefit Over 3 Yr Horizon in NPV Terms =', round(pyomooutput['NPV_Benefit_3YrHorizon'].sum(),2))
print()
print(CAPEX_CheckSum_pyomo.to_string(index = False))

Maximised Benefit Over 3 Yr Horizon in NPV Terms = 53.7

 Year  CAPEX Phasing  Constraints
    1           20.0           20
    2           30.0           30
    3           17.0           25
    4           15.0            0
